# 1. Data prepration
    
## 1.1 Load data and preprocessing
In this part we import libraries and data set. Data set has 9 columns and 2075259 samples. It also has missing values. The first step is to load data set and handle missing values. Also the first two columns contain time and date, which we eliminate in our first version of code.  

In [143]:
import pandas as pd
import numpy as np
import time
import math
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv("household_power_consumption.txt",";")

print("The size of data set is:",data.shape)
print("The first three rows of data set:\n",data.head(3))
print("The last three rows of data set:\n",data.tail(3))

X = pd.DataFrame(data.iloc[:,2:6], columns=["Global_active_power","Global_reactive_power","Voltage","Global_intensity"])
Y = pd.DataFrame(data.iloc[:,7], columns=["Sub_metering_2"])

X = X.replace({'?':0})
Y = Y.replace({'?':0})

/home/shahab/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (2,3,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


The size of data set is: (2075259, 9)
The first three rows of data set:
          Date      Time Global_active_power Global_reactive_power  Voltage  \
0  16/12/2006  17:24:00               4.216                 0.418  234.840   
1  16/12/2006  17:25:00               5.360                 0.436  233.630   
2  16/12/2006  17:26:00               5.374                 0.498  233.290   

  Global_intensity Sub_metering_1 Sub_metering_2  Sub_metering_3  
0           18.400          0.000          1.000            17.0  
1           23.000          0.000          1.000            16.0  
2           23.000          0.000          2.000            17.0  
The last three rows of data set:
                Date      Time Global_active_power Global_reactive_power  \
2075256  26/11/2010  21:00:00               0.938                     0   
2075257  26/11/2010  21:01:00               0.934                     0   
2075258  26/11/2010  21:02:00               0.932                     0   

        Vol

## 1.2 train and test sets
Befor starting the training we need to split our data to train and test. In traditional learning algorithms, people usually use 70%/30% train/test or 60%/20%/20% train/dev/test set. Nowadays, because of having large data sets, people usually use 98%/1%/1% train/dev/test set. In this assignment we used the traditional way of spliting data into train and test with the ratio of 70%/30% of train/test. 

In [3]:
X_features = X.columns
Y_features = Y.columns
XY = pd.concat([X[X_features], Y[Y_features]], axis=1)

# Split XY into training set and test set of equal size
train, test = train_test_split(XY, test_size = 0.30)
# Sort the train and test sets after index (which became unsorted through sampling)
train = train.sort_index(axis=0)
test = test.sort_index(axis=0)

# Extract X,Y components from test and train sets
X_train = train[X_features].astype(float); X_test = test[X_features].astype(float)
Y_train = train[Y_features].astype(float); Y_test = test[Y_features].astype(float)

print("Size of train set (X): ",X_train.shape)
print("Size of train set (Y): ",Y_train.shape)
print("Size of test set (X): ",X_test.shape)
print("Size of test set (Y): ",Y_test.shape)

Size of train set (X):  (1452681, 4)
Size of train set (Y):  (1452681, 1)
Size of test set (X):  (622578, 4)
Size of test set (Y):  (622578, 1)


# 2. Logistic ridge regression with different optimizers

The goal of this part is to implement logistic ridge regression with four different optimizers (GD, SGD, SAG, SVRG). We define logistic ridge as following:  

\begin{equation*}
f(w) = \frac{1}{N} \sum_{i\in[N]} f_i(w) + \lambda ||w||_2^2
\end{equation*}

where 

\begin{equation*}
f_i(w) = \log(1+\exp(y_iw^Tx_i))
\end{equation*}

In the following section we each of optimizers and will try to optimize the specified goal function.

In [12]:
def function_gradient_GD(X, Y, w, lambda_):
    Z = np.matmul(X,np.diagflat(Y))
    D,N = Z.shape
    Z_ = -1 * np.matmul(w.T , Z)
    A = 1/(1+np.exp(Z_))
    A_ = np.diagflat(A-1)
    G = 1 / N * np.sum(np.matmul(Z , A_), axis=1,keepdims=True) + 2 * lambda_ * w
    return G

In [13]:
X = np.array([[2,3,3],[1,4,3]])
Y = np.array([[1],[2],[5]])
w = np.array([[0.01],[0.21]])

print(function_gradient_GD(X,Y,w,0.1))

[[-0.76964991]
 [-0.68160782]]


In [134]:
def solver(x,y, w, alpha = 0.1, num_iters = 1000, lambda_ = 0.1, epsilon = 0.0001, optimizer = "GD"):
    if (optimizer == "GD") :
        for i in range(num_iters):
            g = function_gradient_GD(x, y, w, lambda_)
            w = w - alpha * g
            if (np.linalg.norm(g) <= epsilon):
                break
    elif (optimizer == "SGD"):
        for i in range(num_iters):
            D,N = x.shape
            sample_no = int(np.random.random(1) * N)
            Z = np.matmul(x,np.diagflat(y))
            A = -1 * np.matmul(w.T, Z[:,i])
            G = (Z[:,[i]]*float(1/(1+np.exp(A))-1))+ 2 * lambda_ * w
            w = w - alpha * G
    elif (optimizer == "SVRG"):
        T = 100
        K = math.floor(num_iters/T)
        Z = np.matmul(x,np.diagflat(y))
        N = x.shape[1]
        for k in range(K):
            wz = np.matmul(w.T , Z)
            diag = np.diagflat(1/(1+np.exp(-1*wz))-np.ones((1,N)))
            Ga_ = np.matmul(Z , diag)
            ga_ = (1/N) * np.matmul(Ga_ , np.ones((N,1)))
            for t in range(T):
                r = int(np.random.random(1) * N)
                col = Z[:,[r]]
                #col = col.reshape((col.shape[0],1))
                g = np.matmul(col , (1/(1+np.exp(-1 * np.matmul(w.T , col)))-1))
                Ga_col = Ga_[:,r]
                Ga_col = Ga_col.reshape(Ga_col.shape[0],1)
                w = w - alpha * (g - Ga_col + ga_ + 2 * lambda_ * w)
            
    elif (optimizer == "SAG"):
        Z = np.matmul(x,np.diagflat(y))
        d = x.shape[0]
        N = x.shape[1]
        G = np.zeros((d,N))

        for k in range(num_iters):
            r = int(np.random.random(1) * N)
            col = Z[:,r].reshape(Z[:,r].shape[0],1)
            B = np.matmul(col ,(1/(1 + np.exp(-1 * np.matmul(w.T,col))) - 1))
            B = B.reshape(B.shape[0])
            G[:,r] = B
            g = (1/N) * np.matmul(G , np.ones((N,1)))  + 2 * lambda_ * w
            w = w - alpha * g
            
            
    return w

In [146]:
def cost(x,y,w,lambda_ = 0.01):
    D, N = x.shape
    value = 0
    for i in range(N):
        Z = -1 * y[i] * np.matmul(w.T , (x[:,i]).reshape(D,1))
        value += np.log(1+np.exp(Z))
    norm = np.linalg.norm(w)
    c = lambda_ * norm ** 2
    return value/N + c

In [126]:
X = np.array([[2,3,5],[1,4,4]])
Y = np.array([[1],[2],[7]])
w = np.array([[0.01],[0.21]])

print(cost(X,Y,w,0.1))

[[ 0.25389454]]


In [140]:
y = np.array(Y_train.iloc[0:6000])
x = np.array(X_train.iloc[0:6000,:])

N,D = x.shape
w = np.random.rand(D,1)*0.01

##################GD###################
start = time.time()
gde = solver(x.T,y,w,num_iters=100)
end = time.time()
print("Weights of GD after convergence: \n",gde)

err = cost(x.T,y,gde)
print("Cost of GD after convergence: ",err)

print("Training time for GD: ", end-start)

##################SGD###################
start = time.time()
gde = solver(x.T,y,w, num_iters=500,optimizer = "SGD")
end = time.time()
print("Weights of SGD after convergence: \n",gde)

err = cost(x.T,y,gde)
print("Cost of SGD after convergence: ",err)
print("Training time for SGD: ", end-start)
##################SVRG###################
start = time.time()
gde = solver(x.T,y,w, num_iters=1000,optimizer = "SVRG")
end = time.time()
print("Weights of SVRG after convergence: \n",gde)

err = cost(x.T,y,gde)
print("Cost of SVRG after convergence: ",err)
print("Training time for SVRG: ", end-start)
##################SAG###################
start = time.time()
gde = solver(x.T,y,w, num_iters=100,optimizer = "SAG")
end = time.time()
print("Weights of SAG after convergence: \n",gde)

err = cost(x.T,y,gde)
print("Cost of SAG after convergence: ",err)
print("Training time for SAG: ", end-start)

Weights of GD after convergence: 
 [[  1.39374748e-03]
 [  1.24737032e-04]
 [  1.30549938e-01]
 [  4.42132906e-03]]
Cost of GD after convergence:  [[ 0.47035549]]
Training time for GD:  24.653496980667114
Weights of SGD after convergence: 
 [[  3.99765007e-04]
 [  1.23881580e-05]
 [  3.82345319e-02]
 [  1.69091126e-03]]
Cost of SGD after convergence:  [[ 0.47021625]]
Training time for SGD:  61.7726354598999
Weights of SVRG after convergence: 
 [[  2.59821442e-04]
 [  2.56473699e-05]
 [  3.57130200e-02]
 [  1.10374980e-03]]
Cost of SVRG after convergence:  [[ 0.47022849]]
Training time for SVRG:  1.4326896667480469
Weights of SAG after convergence: 
 [[  1.02091694e-03]
 [  5.44494501e-05]
 [  5.61698936e-02]
 [  2.77872666e-03]]
Cost of SAG after convergence:  [[ 0.4702167]]
Training time for SAG:  0.1322011947631836


In [ ]:
ti= np.zeros((100,4))
cost_= np.zeros((100,4))
for i in range(100):
    ##################GD###################
    start = time.time()
    gde = solver(x.T,y,w,num_iters=i)
    end = time.time()

    cost_[i,0] = cost(x.T,y,gde)

    ti[i,0] = end-start

    ##################SGD###################
    start = time.time()
    gde = solver(x.T,y,w, num_iters=i,optimizer = "SGD")
    end = time.time()

    cost_[i,1] = cost(x.T,y,gde)
    
    ti[i,1] = end-start
    ##################SVRG###################
    start = time.time()
    gde = solver(x.T,y,w, num_iters=i,optimizer = "SVRG")
    end = time.time()

    cost_[i,2] = cost(x.T,y,gde)
    
    ti[i,2] = end-start
    ##################SAG###################
    start = time.time()
    gde = solver(x.T,y,w, num_iters=i,optimizer = "SAG")
    end = time.time()

    cost_[i,3] = cost(x.T,y,gde)
    ti[i,3] = end-start

# results

As we can compare we can see that the GD is better than other algorithms in terms of accuracy but it is slower than other algorithms. GD even with just 100 of iteration take a long time to be trained (24 seconds in comparison with 10 seconds for SGD for example) and interestingly it converges just with these few numbers of iterations. Results shows that SAG is really good in comparision with others. This algorithm is as accurate as others and also it is really fast.

# 3. Tunning the lambda paramter

To tune hyper-paramter there are different methods. One methos is to define space for hyper-parameters of the model, for example here we have a space with just one dimension. Then we can define points in this space and train our model with the paramters of each point. The following image shows how we can define and search space to tune hyper-paramters. Here we consider lambda as a hyper-paramter of the model and then with different values of lambda we compute cost (error metric, it can be MSE, NMAE, or any other metric here for simplicity we used cost which was defined before) for each model. 

![title](search.png)

As the figure shows we can search the space with two different approaches.

- Grid search
- Random search

Both of these methods are very common but the Random seach in first round is better. Because in this search it is mostly probable to find a good solution. We can also use our knowledge to define the space, for example we can define a logarithmic scale for our space. We tune this hyper-paramter just for the logistic ridge regression with SAG optimizer. 

In [ ]:
space = [0.01, 0.1, 1, 10, 100, 1000]

for la in space:
    start = time.time()
    gde = solver(x.T,y,w, num_iters=100,optimizer = "SAG", lambda_=la)
    end = time.time()
    print("Weights of SAG after convergence: \n",gde)

    err = cost(x.T,y,gde)
    print("Cost of SAG after convergence: ",err)
    print("Training time for SAG: ", end-start)